# 마이데이터 투자탭 데모 

## Import libraries

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly

import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler, PowerTransformer

import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

import pickle

In [2]:
import FinanceDataReader as fdr
import quantstats as qs

In [3]:
## custom libs

from korquanttools.pricevolume.loader import KRXPriceDM
from korquanttools.pricevolume.utils import DateUtil
from korquanttools.pricevolume.config import PathConfig

In [31]:
CWD = Path('.').resolve()
WOORI_DATA_PATH = CWD / 'woori_data'

## Import dataset

In [56]:
# Global parameters

START = 20140101
END = 20221202

WINDOW = 252 # trading day 기준
# WINDOW = 60 # trading day 기준
# WINDOW = 20 # trading day 기준

In [57]:
# Init data module
pricevolume = KRXPriceDM(START, END)
pricevolume.get_info()


        * DM name: KRX_pricevolume
        * DM description: Basic price-volume data imported from KRX website & NAVER finance. Has KOSPI, KOSDAQ, KONEX stocks.
        * birthday: 20211203
        * DM period: 19990101 ~ 
        * Available data: ['lv1', 'open', 'high', 'low', 'close', 'volume', 'dollarvolume', 'marketcap']
        


### 수익률

In [58]:
## stock return
return_df = pd.read_pickle(PathConfig.cache_path / f"temp_return_{START}_{END}")

In [59]:
holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

tradingdays

DatetimeIndex(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07',
               '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-13',
               '2014-01-14', '2014-01-15',
               ...
               '2022-11-21', '2022-11-22', '2022-11-23', '2022-11-24',
               '2022-11-25', '2022-11-28', '2022-11-29', '2022-11-30',
               '2022-12-01', '2022-12-02'],
              dtype='datetime64[ns]', name='trdDd', length=2190, freq=None)

In [60]:
return_df = return_df.loc[tradingdays, :].copy()

### 수익률 통계량 확인 

- 업종 서브페이지에서 어느 정도를 '보합'으로 잡을지 판단하기 위해 확인

In [61]:
pd.DataFrame(return_df.to_numpy().flatten() * 100).describe()

,0
count,5.081375e+06
mean,5.401327e-02
std,3.832073e+00
min,-9.969191e+01
25%,-1.330377e+00
50%,0.000000e+00
75%,1.129235e+00
max,1.605686e+03


### 시가총액

In [62]:
# Data for 2nd Factor: SMB (Small Minus Big)

marcap_df = pricevolume.get_data("marketcap")

In [63]:
marcap_df = marcap_df.astype(float) # object로 되어있었음

In [64]:
marcap_df = marcap_df.loc[tradingdays, :].copy()

### Grouping

In [65]:
with open('INDUSTRY_NAME2CODE.pickle', 'rb') as handle:
    INDUSTRY_NAME2CODE = pickle.load(handle)

In [66]:
INDUSTRY_CODE2NAME = {v:k for k,v in INDUSTRY_NAME2CODE.items()}

In [68]:
ksic_df = pd.read_excel(WOORI_DATA_PATH / 'ksic_lv1lv2.xlsx', skiprows=2)
ksic_df = ksic_df.ffill()
ksic_df = ksic_df.drop_duplicates()
# ksic_df.to_excel('ksic.xlsx')

KRX 산업분류는 표준산업분류를 기초로 했다고는 하나 굉장히 근본없이 한거같다. 제대로 된 업종 분류로 보이지 않음. 

대분류와 중분류가 섞여있는 모습을 보이고, 서비스업 등은 무엇을 지칭하는 것인지 불분명하다. 

KRX에 설명을 적어놓지도 않았다. 이런... 

그래서 아래와 같이 일단 내가 직접 노가다로 대분류 표준산업분류와 매핑을 시켜봤다. 

 0: '서비스업', -->  전문, 과학 및 기술 서비스업(70~73)

 1: '기타금융', --> 금융 및 보험업(64~66)

 2: '섬유의복', --> 제조업(10~34)

 3: '운수창고업', --> 운수 및 창고업(49~52)

 4: '음식료품', --> 제조업(10~34)

 5: '화학', --> 제조업(10~34)

 6: '철강금속', --> 제조업(10~34)

 7: '유통업', --> 운수 및 창고업(49~52)

 8: '건설업', --> 건설업(41~42)

 9: '증권', --> 금융 및 보험업(64~66)

 10: '전기전자', --> 제조업(10~34)

 11: '의약품', --> 제조업(10~34)

 12: '기계', --> 제조업(10~34)

 13: '종이목재', --> 제조업(10~34)

 14: '통신업', --> 정보통신업(58~63)

 15: '기타제조업', --> 제조업(10~34)

 16: '보험', --> 금융 및 보험업(64~66)

 17: '운수장비', --> 제조업(10~34)

 18: '전기가스업', --> 전기, 가스, 증기 및 공기 조절 공급업(35)

 19: '비금속광물', --> 광업(05~08)

 20: '은행', --> 금융 및 보험업(64~66)

 21: '광업', --> 광업(05~08)

 22: '농업, 임업 및 어업', --> 농업, 임업 및 어업(01~03)

 23: '의료정밀', --> 제조업(10~34)

 24: '반도체', --> 제조업(10~34)

 25: '금속', --> 제조업(10~34)

 26: '기타서비스', --> 전문, 과학 및 기술 서비스업(70~73)

 27: '방송서비스', --> 정보통신업(58~63)

 28: '유통', --> 운수 및 창고업(49~52)

 29: '제약', --> 제조업(10~34)

 30: '통신장비', --> 제조업(10~34)

 31: 'IT부품', --> 제조업(10~34)

 32: '기계·장비', --> 제조업(10~34)

 33: '오락·문화', --> 예술, 스포츠 및 여가관련 서비스업(90~91)

 34: '운송장비·부품', --> 제조업(10~34)

 35: '건설', --> 건설업(41~42)

 36: '전기·가스·수도', --> 전기, 가스, 증기 및 공기 조절 공급업(35)

 37: '소프트웨어', --> 정보통신업(58~63)

 38: '인터넷', --> 정보통신업(58~63)

 39: '디지털컨텐츠', --> 정보통신업(58~63)

 40: '출판·매체복제', --> 정보통신업(58~63)

 41: '금융', --> 금융 및 보험업(64~66)

 42: '운송', --> 운수 및 창고업(49~52)

 43: '통신서비스', --> 정보통신업(58~63)

 44: '일반전기전자', --> 제조업(10~34)

 45: '섬유·의류', --> 제조업(10~34)

 46: '컴퓨터서비스', --> 정보통신업(58~63)

 47: '음식료·담배', --> 제조업(10~34)

 48: '비금속', --> 제조업(10~34)

 49: '종이·목재', --> 제조업(10~34)

 50: '의료·정밀기기', --> 제조업(10~34)

 51: '기타제조', --> 제조업(10~34)

 52: '정보기기', --> 제조업(10~34)

 53: '숙박·음식' --> 숙박 및 음식점업(55~56)

In [69]:
CODE2KSIC_MAPPING = {
    0: '전문, 과학 및 기술 서비스업(70~73)',
    1: '금융 및 보험업(64~66)',
    2: '제조업(10~34)',
    3: '운수 및 창고업(49~52)',
    4: '제조업(10~34)',
    5: '제조업(10~34)',
    6: '제조업(10~34)',
    7: '운수 및 창고업(49~52)',
    8: '건설업(41~42)',
    9: '금융 및 보험업(64~66)',
    10: '제조업(10~34)',
    11: '제조업(10~34)',
    12: '제조업(10~34)',
    13: '제조업(10~34)',
    14: '정보통신업(58~63)',
    15: '제조업(10~34)',
    16: '금융 및 보험업(64~66)',
    17: '제조업(10~34)',
    18: '전기, 가스, 증기 및 공기 조절 공급업(35)',
    19: '광업(05~08)',
    20: '금융 및 보험업(64~66)',
    21: '광업(05~08)',
    22: '농업, 임업 및 어업(01~03)',
    23: '제조업(10~34)',
    24: '제조업(10~34)',
    25: '제조업(10~34)',
    26: '전문, 과학 및 기술 서비스업(70~73)',
    27: '정보통신업(58~63)',
    28: '운수 및 창고업(49~52)',
    29: '제조업(10~34)',
    30: '제조업(10~34)',
    31: '제조업(10~34)',
    32: '제조업(10~34)',
    33: '예술, 스포츠 및 여가관련 서비스업(90~91)',
    34: '제조업(10~34)',
    35: '건설업(41~42)',
    36: '전기, 가스, 증기 및 공기 조절 공급업(35)',
    37: '정보통신업(58~63)',
    38: '정보통신업(58~63)',
    39: '정보통신업(58~63)',
    40: '정보통신업(58~63)',
    41: '금융 및 보험업(64~66)',
    42: '운수 및 창고업(49~52)',
    43: '정보통신업(58~63)',
    44: '제조업(10~34)',
    45: '제조업(10~34)',
    46: '정보통신업(58~63)',
    47: '제조업(10~34)',
    48: '제조업(10~34)',
    49: '제조업(10~34)',
    50: '제조업(10~34)',
    51: '제조업(10~34)',
    52: '제조업(10~34)',
    53: '숙박 및 음식점업(55~56)',
    }

In [70]:
industry_df = pd.read_pickle('krx_industry_df_20140101_20221202.pickle')

In [71]:
with open('sid2name.pkl', 'rb') as p:
    sid2name = pickle.load(p)

## 업종별 수익률

In [109]:
# DATE_BEFORE = -1

DATE = '2022-05-10'

In [110]:
last_return_v = return_df.loc[DATE, :]

last_marcap_v = marcap_df.loc[DATE, :]
last_group_v = industry_df.loc[DATE, :]

In [111]:
last_group_v.isna().sum()

613

In [112]:
last_lv1_df = pd.DataFrame(data={
    'ret': last_return_v,
    'marcap': last_marcap_v,
    'group_idx': last_group_v,
})

In [113]:
len(last_lv1_df)

3110

In [114]:
last_lv1_df = last_lv1_df.dropna()
last_lv1_df = last_lv1_df.reset_index()
last_lv1_df

,ISU_SRT_CD,ret,marcap,group_idx
0,000020,-0.004255,3.267982e+11,11.0
1,000040,-0.009756,7.806376e+10,17.0
2,000050,-0.009646,4.221952e+11,7.0
3,000060,-0.032941,4.957688e+12,16.0
4,000070,-0.022864,6.954188e+11,1.0
...,...,...,...,...
2492,405640,-0.002268,9.977000e+09,41.0
2493,412930,0.002278,1.358720e+10,41.0
2494,413600,-0.009259,9.175250e+09,41.0
2495,415580,-0.004608,1.146960e+10,41.0


In [115]:
## 표준산업분류로 mapping, 더 이상 idx가 아님. string(object) 임.

last_lv1_df['group_idx'] = last_lv1_df['group_idx'].apply(lambda g: CODE2KSIC_MAPPING[g])

In [116]:
group_marcap_weight_df = last_lv1_df[['group_idx', 'marcap']].groupby(by='group_idx').sum()
group_marcap_weight_df.reset_index(drop=False, inplace=True)

group_marcap_weight_df.rename(columns={'marcap': 'group_marcap_sum'}, inplace=True)

In [117]:
KSIC2GRPMARCAP = {k:v for k, v in group_marcap_weight_df.values}
KSIC2GRPMARCAP

{'건설업(41~42)': 26198153562911.0,
 '광업(05~08)': 22016484273354.0,
 '금융 및 보험업(64~66)': 286160436212795.0,
 '농업, 임업 및 어업(01~03)': 1646800025400.0,
 '숙박 및 음식점업(55~56)': 1332430883950.0,
 '예술, 스포츠 및 여가관련 서비스업(90~91)': 14035161673064.0,
 '운수 및 창고업(49~52)': 140819542641616.0,
 '전기, 가스, 증기 및 공기 조절 공급업(35)': 22190285003505.0,
 '전문, 과학 및 기술 서비스업(70~73)': 233379486585672.0,
 '정보통신업(58~63)': 83672094817526.0,
 '제조업(10~34)': 1592704830290189.0}

In [118]:
last_lv1_df.loc[:, 'group_marcap_sum'] = last_lv1_df['group_idx'].apply(lambda g_nm: KSIC2GRPMARCAP[g_nm])

In [119]:

last_lv1_df['group_marcap_weight'] = last_lv1_df['marcap'] / last_lv1_df['group_marcap_sum']
last_lv1_df['group_weighted_ret'] = last_lv1_df['ret'] * last_lv1_df['group_marcap_weight']


In [120]:
group_tree_df = last_lv1_df[['group_idx', 'group_weighted_ret', 'marcap']].groupby(by='group_idx').sum()
group_tree_df.reset_index(drop=False, inplace=True)

In [121]:
group_tree_df['group_weighted_ret'] = 100 * group_tree_df['group_weighted_ret']

In [122]:
group_tree_df.columns

Index(['group_idx', 'group_weighted_ret', 'marcap'], dtype='object')

In [123]:
group_tree_df['log_marcap'] = group_tree_df['marcap'].apply(np.log)

In [124]:
scaler = RobustScaler()

group_tree_df['robust_marcap'] = scaler.fit_transform(group_tree_df['marcap'].to_frame())
# group_tree_df['robust_marcap'] = group_tree_df['robust_marcap'] + abs(group_tree_df['robust_marcap'].min())

In [125]:
scaler = PowerTransformer(method='yeo-johnson')

group_tree_df['power_marcap'] = scaler.fit_transform(group_tree_df['marcap'].to_frame())

In [126]:
group_tree_df['sqrt_marcap'] = group_tree_df['marcap'].apply(np.sqrt)

In [134]:
chart_name = '업종별 수익률 현황'

treemap_fig = px.treemap(
    group_tree_df, 
    title=chart_name,
    path=[
        px.Constant('KOSPI & KOSDAQ'),
        'group_idx',
    ],
    # values='power_marcap',
    values='marcap',
    color='group_weighted_ret',
    # hover_data=[],
    # color_discrete_sequence=,
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
)

treemap_fig.update_layout(
    margin=dict(t=50, l=25, b=25),
    width=300+150,
    height=400,
    legend=dict(orientation='h'),
    )
treemap_fig.show()
